In [5]:
import re

In [11]:
with open("example_input.txt", 'r') as IN:
    lines = IN.readlines()


SMATRIX = open('S_matrix.txt', 'a')
COMPOUNDS = open('compounds-1.txt', 'a')
RXN = open('reactions.txt', 'a')
RRXN = open('reversible_reactions.txt', 'a')
UPPBOUND = open('upperbound_on_fluxes.txt', 'a')
LOWBOUND = open('lowerbound_on_fluxes.txt', 'a')

SMATRIX.write("/\n")
RXN.write("/\n")
RRXN.write("/\n")
UPPBOUND.write("/\n")
LOWBOUND.write("/\n")

2

In [20]:
for line in lines:
    line = line.strip()
    fields = line.split(";")
    if fields[3]:
        print("compartment defined")
    else:
        print("compartment not defined")

    comp = fields[3].split(',')

    for c in comp:
        RXN.write(f"'{fields[0]}[{c}]'\n")
        #print(f"RXN: '{fields[0]}[{c}]'\n")

        
    # 2. Now analyze everything to generate the matrix and the list of compounds
    if "<=>" in fields[1]:
        #print("reversible reaction")
        met = fields[1].split(" <=> ")
        met_izq = met[0].split(" + ")
        met_der = met[1].split(" + ")
        print("METDER", met_der)
    else:
        #print("irreversible reaction")
        met = fields[1].split("->")
        met_izq = met[0].split(" + ")
        met_der = met[1].split(" + ")
        #print(met_izq, met_der)
    
    if "<=>" in fields[1]:
        for c in comp:
            RRXN.write(f"'{fields[0]}[{c}]'\n")
            #print(f"RRXN: '{fields[0]}[{c}]'\n")

        for met_izq_2 in met_izq:
            if re.search(r'^\s?(\d+\.\d+|\d+)\s', met_izq_2):
                coef = re.findall(r'^\s?(\d+\.\d+|\d+)\s', met_izq_2)[0]
                compound = met_izq_2.split()[1]
                for c in comp:
                    #print(f"COMPOUNDS_IZQ: '{compound}[{c}]\n'")
                    #print(f"SMATRIX_IZQ: '{compound}[{c}]'.'{fields[0]}[c]' -{coef}\n")
                    COMPOUNDS.write(f"'{compound}[{c}]'\n")
                    SMATRIX.write(f"'{compound}[{c}]'.'{fields[0]}[{c}]' -{coef}\n")

        for met_der_2 in met_der:
            if re.search(r'^\s?(\d+\.\d+|\d+)\s', met_der_2):
                coef = re.findall(r'^\s?(\d+\.\d+|\d+)\s', met_der_2)[0]
                compound = met_der_2.split()[1]
                for c in comp:
                    #print(f"COMPOUNDS_DER: '{compound}[{c}]\n'")
                    #print(f"SMATRIX_DER: '{compound}[{c}]'.'{fields[0]}[c]' {coef}\n")
                    COMPOUNDS.write(f"'{compound}[{c}]'\n")
                    SMATRIX.write(f"'{compound}[{c}]'.'{fields[0]}[{c}]' -{coef}\n")

        # Printing reaction limits
        for c in comp:
            #print(f"'{fields[0]}[{c}]' 1000\n")
            #print(f"'{fields[0]}[{c}]' -1000\n")
            UPPBOUND.write(f"'{fields[0]}[{c}]' 1000\n")
            LOWBOUND.write(f"'{fields[0]}[{c}]' -1000\n")
        
    elif "->" in fields[1]:  # Evaluating Forward
        for met_izq_2 in met_izq:
            if re.search(r'^\s?(\d+\.\d+|\d+)\s', met_izq_2):
                coef = re.findall(r'^\s?(\d+\.\d+|\d+)\s', met_izq_2)[0]
                compound = met_izq_2.split()[1]
                print(f"'{compound}[{c}]'\n")
                for c in comp:
                    COMPOUNDS.write(f"'{compound}[{c}]'\n")
                    SMATRIX.write(f"'{compound}[{c}]'.'{fields[0]}[c]' -{coef}\n")
        for met_der_2 in met_der:
            if re.search(r'^\s?(\d+\.\d+|\d+)\s', met_der_2):
                coef = re.findall(r'^\s?(\d+\.\d+|\d+)\s', met_der_2)[0]
                compound = met_izq_2.split()[1]
                for c in comp:
                    COMPOUNDS.write(f"'{compound}[{c}]'\n")
                    SMATRIX.write(f"'{compound}[{c}]'.'{fields[0]}[c]' -{coef}\n")

        for c in comp:
            #print(f"'{fields[0]}[{c}]' 1000\n")
            #print(f"'{fields[0]}[{c}]' 0\n")
            UPPBOUND.write(f"'{fields[0]}[{c}]' 1000\n")
            LOWBOUND.write(f"'{fields[0]}[{c}]' 0\n")

compartment defined
METDER ['Met3', '7 Met4']
Met3
7 Met4
compartment defined
'Met5[MNXC1]'

'Met6[MNXC1]'

compartment defined
METDER ['Met11', 'Met12']
Met11
Met12


In [22]:
import re

reaction = "ACP_c[c] + ATP_c[c] + CPD-17686_c[c]  <=> _5-hexenoyl-ACPs_c[c] + AMP_c[c] + PPI_c[c]"

# Split the reaction into the left-hand side and right-hand side
lhs, rhs = reaction.split('<=>')

# Extract coefficients and compounds from the left-hand side
lhs_compounds = re.findall(r'(\d*\.\d+|\d+)\s*([^\s+\-]+(?:-[^\s+]+)?)', lhs)

# Extract coefficients and compounds from the right-hand side
rhs_compounds = re.findall(r'(\d*\.\d+|\d+)\s*([^\s+\-]+(?:-[^\s+]+)?)', rhs)

# Print the parsed reaction
print("Left-hand side:")
for coefficient, compound in lhs_compounds:
    print(f"Coefficient: {coefficient}, Compound: {compound}")

print("Right-hand side:")
for coefficient, compound in rhs_compounds:
    print(f"Coefficient: {coefficient}, Compound: {compound}")


Left-hand side:
Coefficient: 17686, Compound: _c[c]
Right-hand side:


In [9]:
SMATRIX.write("/\n")
RXN.write("/\n")
RRXN.write("/\n")
UPPBOUND.write("/\n")
LOWBOUND.write("/\n")

SMATRIX.close()
COMPOUNDS.close()
RXN.close()
RRXN.close()
UPPBOUND.close()
LOWBOUND.close()   


In [26]:
with open("example_input.txt", 'r') as IN:
    lines = IN.readlines()
    print(lines)


SMATRIX = open('S_matrix.txt', 'a')
COMPOUNDS = open('compounds-1.txt', 'a')
RXN = open('reactions.txt', 'a')
RRXN = open('reversible_reactions.txt', 'a')
UPPBOUND = open('upperbound_on_fluxes.txt', 'a')
LOWBOUND = open('lowerbound_on_fluxes.txt', 'a')

SMATRIX.write("/\n")
RXN.write("/\n")
RRXN.write("/\n")
UPPBOUND.write("/\n")
LOWBOUND.write("/\n")

for line in lines:
    line = line.strip()
    fields = line.split("\t")
    print(fields)

    if fields[3]:
        print("compartment defined")
    else:
        print("compartment not defined")
        fields[3] = 'MNXC1'  # If an assignation for 'cell' compartment is desired.

    comp = fields[3].split(',')

    for c in comp:
        RXN.write(f"'{fields[0]}@{c}'\n")

    # 2. Now analyze everything to generate the matrix and the list of compounds

    met = fields[1].split(" = ")
    met_izq = met[0].split(" + ")
    met_der = met[1].split(" + ")

    if fields[2] or fields[2] == 0:
        print("dG is defined")
    else:
        print("dG is not defined")
        fields[2] = -1  # Assign a negative value to reactions without dG to make them Forward

    if fields[2] == 0:  # Evaluating reversibles
        for c in comp:
            RRXN.write(f"'{fields[0]}@{c}'\n")

        for met_izq_2 in met_izq:
            if re.search(r'^\s?(\d+\.\d+|\d+)\s', met_izq_2):
                coef = re.findall(r'^\s?(\d+\.\d+|\d+)\s', met_izq_2)[0]
                for c in comp:
                    COMPOUNDS.write(f"'{met_izq_2}@{c}'\n")
                    SMATRIX.write(f"'{met_izq_2}@{c}'.'{fields[0]}@{c}' -{coef}\n")

        for met_der_2 in met_der:
            if re.search(r'^\s?(\d+\.\d+|\d+)\s', met_der_2):
                coef = re.findall(r'^\s?(\d+\.\d+|\d+)\s', met_der_2)[0]
                for c in comp:
                    COMPOUNDS.write(f"'{met_der_2}@{c}'\n")
                    SMATRIX.write(f"'{met_der_2}@{c}'.'{fields[0]}@{c}' -{coef}\n")

        # Printing reaction limits
        for c in comp:
            UPPBOUND.write(f"'{fields[0]}@{c}' 1000\n")
            LOWBOUND.write(f"'{fields[0]}@{c}' -1000\n")

    elif fields[2] < 0:  # Evaluating Forward
        for met_izq_2 in met_izq:
            if re.search(r'^\s?(\d+\.\d+|\d+)\s', met_izq_2):
                coef = re.findall(r'^\s?(\d+\.\d+|\d+)\s', met_izq_2)[0]
                for c in comp:
                    COMPOUNDS.write(f"'{met_izq_2}@{c}'\n")
                    SMATRIX.write(f"'{met_izq_2}@{c}'.'{fields[0]}@{c}' -{coef}\n")

        for met_der_2 in met_der:
            if re.search(r'^\s?(\d+\.\d+|\d+)\s', met_der_2):
                coef = re.findall(r'^\s?(\d+\.\d+|\d+)\s', met_der_2)[0]
                for c in comp:
                    COMPOUNDS.write(f"'{met_der_2}@{c}'\n")
                    SMATRIX.write(f"'{met_der_2}@{c}'.'{fields[0]}@{c}' {coef}\n")

        for c in comp:
            UPPBOUND.write(f"'{fields[0]}@{c}' 1000\n")
            LOWBOUND.write(f"'{fields[0]}@{c}' 0\n")

    elif fields[2] > 0:  # Evaluating Backward
        for met_izq_2 in met_izq:
            if re.search(r'^\s?(\d+\.\d+|\d+)\s', met_izq_2):
                coef = re.findall(r'^\s?(\d+\.\d+|\d+)\s', met_izq_2)[0]
                for c in comp:
                    COMPOUNDS.write(f"'{met_izq_2}@{c}'\n")
                    SMATRIX.write(f"'{met_izq_2}@{c}'.'{fields[0]}@{c}' {coef}\n")

        for met_der_2 in met_der:
            if re.search(r'^\s?(\d+\.\d+|\d+)\s', met_der_2):
                coef = re.findall(r'^\s?(\d+\.\d+|\d+)\s', met_der_2)[0]
                for c in comp:
                    COMPOUNDS.write(f"'{met_der_2}@{c}'\n")
                    SMATRIX.write(f"'{met_der_2}@{c}'.'{fields[0]}@{c}' -{coef}\n")

        for c in comp:
            UPPBOUND.write(f"'{fields[0]}@{c}' 0\n")
            LOWBOUND.write(f"'{fields[0]}@{c}' -1000\n")
SMATRIX.write("/\n")
RXN.write("/\n")
RRXN.write("/\n")
UPPBOUND.write("/\n")
LOWBOUND.write("/\n")

SMATRIX.close()
COMPOUNDS.close()
RXN.close()
RRXN.close()
UPPBOUND.close()
LOWBOUND.close()   


['Reaction1    Met1 + Met2 = Met3 + Met4    -2.5    MNXC1,MNXC2,MNXC3\n', 'Reaction2    Met5 + Met6 = Met7 + Met8    0        MNXC1,MNXC4\n', 'Reaction3    Met9 + Met10 = Met11 + Met12  1.8      MNXC2,MNXC4,MNXC5']
['Reaction1    Met1 + Met2 = Met3 + Met4    -2.5    MNXC1,MNXC2,MNXC3']


IndexError: list index out of range